# Ticket Tracker - Reduzindo o sofrimento da compra de ingressos

## Índice

## Introdução

## Problema 

## Extração de dados 

In [43]:
# Selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Beautiful Soup imports
import bs4
from bs4 import BeautifulSoup

import utils
import random
import requests
import time
import re


In [2]:
# Creating a selenium driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.set_page_load_timeout(60)

In [3]:
# Accessing blacktag
url = 'https://blacktag.com.br/eventos?cidade=S%C3%A3o+Paulo'

# Using random user agents to avoid host suspition
u_agent = random.choice(utils.user_agents)

arguments = [
    "--disable-cookies",
    "--disable-local-storage",
    "--disable-session-storage",
    "--block-third-party-cookies"
    f"user-agent={u_agent}"
]

chrome_options = Options()
for arg in arguments:
    chrome_options.add_argument(arg)

driver = webdriver.Chrome(options=chrome_options)

driver.get(url=url)
driver.implicitly_wait(10)

In [5]:
timeout = 10
wait = WebDriverWait(driver, timeout)

button = driver.find_element(By.ID, 'load-more-events')
while button.is_displayed():
    button.click()

ElementClickInterceptedException: Message: element click intercepted: Element <button data-page="1" data-search="" class="btn py-2 btn-primary text-uppercase font-weight-bold btn-block loading" id="load-more-events" data-url="/eventos?cidade=S%C3%A3o+Paulo" disabled="" style="display: none;">...</button> is not clickable at point (345, 19). Other element would receive the click: <p class="card-text text-dark">...</p>
  (Session info: chrome=124.0.6367.91)
Stacktrace:
#0 0x62dc378c7cb3 <unknown>
#1 0x62dc375b64a7 <unknown>
#2 0x62dc37607f73 <unknown>
#3 0x62dc37605e76 <unknown>
#4 0x62dc37603847 <unknown>
#5 0x62dc37602c14 <unknown>
#6 0x62dc375f61c9 <unknown>
#7 0x62dc37623412 <unknown>
#8 0x62dc375f5b58 <unknown>
#9 0x62dc376235de <unknown>
#10 0x62dc37641be9 <unknown>
#11 0x62dc376231b3 <unknown>
#12 0x62dc375f424a <unknown>
#13 0x62dc375f4c1e <unknown>
#14 0x62dc3788bfdb <unknown>
#15 0x62dc3788ff2b <unknown>
#16 0x62dc378780f1 <unknown>
#17 0x62dc37890a92 <unknown>
#18 0x62dc3785cf

In [18]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

tags = soup.find_all("a", class_ = "w-100")

events = ["https://blacktag.com.br" + tag.get("href") + "/ingressos" for tag in tags]
events

['https://blacktag.com.br/eventos/20397/fusion/ingressos',
 'https://blacktag.com.br/eventos/20180/deixa-em-off-x-festa-do-chefe/ingressos',
 'https://blacktag.com.br/eventos/20557/sextou-open-bar-premium-03-05/ingressos',
 'https://blacktag.com.br/eventos/20363/advogado-do-diabo/ingressos',
 'https://blacktag.com.br/eventos/20818/apagao-no-escurinho-voce-se-solta/ingressos',
 'https://blacktag.com.br/eventos/20416/baile-de-gala-unificados-4116-4141-4155-4162-4167/ingressos',
 'https://blacktag.com.br/eventos/20339/zodiaco-os-astros-se-alinharam/ingressos',
 'https://blacktag.com.br/eventos/20553/interclasses-medicina-sao-caetano/ingressos',
 'https://blacktag.com.br/eventos/19469/aniversario-do-interatleticas/ingressos',
 'https://blacktag.com.br/eventos/20641/ressaca-pos-provas/ingressos',
 'https://blacktag.com.br/eventos/20799/open-sunset-04-05-open-bar/ingressos',
 'https://blacktag.com.br/eventos/20340/cervejada-pre-juca/ingressos',
 'https://blacktag.com.br/eventos/20603/baile-d

In [54]:
event_info = { index: {"url": url} for index, url in enumerate(events[:2])}
print(event_info)

{0: {'url': 'https://blacktag.com.br/eventos/20397/fusion/ingressos'}, 1: {'url': 'https://blacktag.com.br/eventos/20180/deixa-em-off-x-festa-do-chefe/ingressos'}}


In [70]:
import time

for index, url in enumerate(events[:2]):
    time.sleep(random.uniform(0.5, 1.5))    

    headers = {
        'User-Agent': random.choice(utils.user_agents),
    }

    params = {
        'localStorageEnabled': 'false',
        'sessionStorageEnabled': 'false',
    }
    
    response = requests.get(url, headers=headers, cookies={}, params=params)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    a_elements = soup.find("a", class_ = "text-dark")
    event_name = a_elements.text.strip()
    
    p_elements = soup.find_all("p", class_ = "text-primary mb-sm-2 mb-1")
    event_date = p_elements[0].text.strip()
    event_time = p_elements[1].text.strip()

    cols = soup.find_all("div", class_ = "col py-2 px-4")
    
    tickets = dict()

    for ticket in cols:
        ticket_type = ticket.find("div", class_ = "ml-md-2").text.strip()
        lote = ticket.find("div", class_ = "col-md-3 col-12 allotment").text.strip()
        lote_n = re.findall(r'\d+', lote)
        if lote_n:
            lote_n = lote_n[0]
        else:
            lote_n = lote
        
        price = ticket.find("div", class_ ="col-md-3 col-12 price").text.strip()
        price_n = re.findall(r'\d+,\d+', price)[0]
        tickets[ticket_type] = dict()
        tickets[ticket_type]["lot"] = lote_n
        tickets[ticket_type]["price"] = price_n

    event_info[index]["name"] = event_name     
    event_info[index]["date"] = event_date
    event_info[index]["time"] = event_time
    event_info[index]["tickets"] = tickets    

https://blacktag.com.br/eventos/20397/fusion/ingressos
https://blacktag.com.br/eventos/20180/deixa-em-off-x-festa-do-chefe/ingressos


In [71]:
event_info

{0: {'url': 'https://blacktag.com.br/eventos/20397/fusion/ingressos',
  'name': 'Fusion',
  'date': 'Sex 03 de Mai',
  'time': '20:00',
  'tickets': {'Associado': {'lot': '2', 'price': '85,00'},
   'Aluno Einstein': {'lot': '2', 'price': '95,00'},
   'Externo': {'lot': '2', 'price': '100,00'}}},
 1: {'url': 'https://blacktag.com.br/eventos/20180/deixa-em-off-x-festa-do-chefe/ingressos',
  'name': 'Deixa em Off x Festa do Chefe',
  'date': 'Sex 03 de Mai',
  'time': '21:00',
  'tickets': {'PISTA OPEN BAR': {'lot': '4', 'price': '130,00'},
   'ÁREA VIP OPEN BAR': {'lot': '4', 'price': '215,00'},
   'FRONTSTAGE SECO': {'lot': '2', 'price': '100,00'},
   'PISTA OPEN BAR - PROMOÇÃO 2 INGRESSOS': {'lot': 'Promocional',
    'price': '120,00'},
   'PISTA OPEN BAR - PROMOÇÃO 3 INGRESSOS': {'lot': 'Promocional',
    'price': '110,00'},
   'ÁREA VIP OPEN BAR - PROMOÇÃO 2 INGRESSOS': {'lot': 'Promocional',
    'price': '200,00'},
   'ÁREA VIP OPEN BAR - PROMOÇÃO 3 INGRESSOS': {'lot': 'Promocional'

## Análise

## Conclusão